# Loading more information about the hotels

## Extract data from the metadata table in the sqlite database

In [1]:
# Dependencies
import pandas as pd
import sqlite3
from IPython.core.display import clear_output

In [2]:
# Create a connection to the database
conn = sqlite3.connect("Data/Hotels.db")

# Load the database table into a pandas dataframe
metadata = pd.read_sql_query("select * from metadata;", conn)
conn.close()

# Preview the dataframe
metadata.head()

,index,name,categories,primaryCategories,address,city,province,latitude,longitude,websites
0,0,Rancho Valencia Resort Spa,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,5921 Valencia Cir,Rancho Santa Fe,CA,32.990959,-117.186136,http://www.ranchovalencia.com
1,3,Aloft Arundel Mills,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,7520 Teague Rd,Hanover,MD,39.155929,-76.716341,http://www.starwoodhotels.com/alofthotels/prop...
2,9,Hampton Inn Suites PortlandVancouver,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,315 SE Olympia Dr,Vancouver,WA,45.619212,-122.525196,http://hamptoninn3.hilton.com/en/hotels/washin...
3,15,Hotel Phillips,"Hotels,Caterers,Hotels and motels,Hotel,Restau...",Accommodation & Food Services,106 W 12th St,Kansas City,MO,39.100119,-94.584701,http://curiocollection3.hilton.com/en/hotels/m...
4,20,The Inn at Solvang,"Bed Breakfasts,Bed Breakfast,Hotels Motels,Hotel",Accommodation & Food Services,10611 Standing Stone Rd,Huntingdon,PA,40.527478,-77.969763,http://www.solvang.com


## Add hotel characteristics

### Distance from airport

In [3]:
# Get latitudes and longitudes of airports mapped in the USA
# Source: https://opendata.socrata.com/dataset/Airport-Codes-mapped-to-Latitude-Longitude-in-the-/rxrh-4cxm
path = "Data/Airport_Codes_Coords_USA.csv"

airports = pd.read_csv(path)
airports.head()

,locationID,Latitude,Longitude
0,ADK,51.8781,176.6461
1,AKK,56.9386,154.1825
2,Z13,60.9047,161.4225
3,AKI,60.9028,161.2306
4,AUK,62.6800,164.6600


In [4]:
print(f"Number of airports mapped in USA: {airports.shape[0]}")

Number of airports mapped in USA: 13429


In [5]:
loc = {"lat": metadata["latitude"][0], "lon": metadata["longitude"][0]}
loc

{'lat': 32.990959000000004, 'lon': -117.186136}

In [6]:
# Get a list of airport latitudes
airport_lat = [i for i in airports["Latitude"]]
airport_lon = [j for j in airports["Longitude"]]

In [7]:
# Haversine formula to calculate distance
# Source1: https://stackoverflow.com/a/41337005
# Source2: https://stackoverflow.com/a/21623206
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    """distance is expressed in km"""
    p = 0.017453292519943295 # pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) # Earth diameter: 12742 = 2 * R; R = 6371km (mean radius of the earth)

# Find the minimum distance between the hotel and the closest airport
def min_distance(lat, lon):
    distances = []
    for i in range(0, len(airports)):
        away = distance(abs(lat), abs(lon), abs(airports["Latitude"][i]), abs(airports["Longitude"][i]))
        distances.append(away)
    return min(distances)

In [8]:
min_distance(loc["lat"], loc["lon"])

14.308848055373579

In [9]:
# For each hotel coordinate, calculate the distance to the nearest airport
airport_distance = []
for i in range(0, len(metadata)):
    dist = min_distance(metadata["latitude"][i], metadata["longitude"][i])
    airport_distance.append(dist)
    
    print(f"Now processing {i}th airport.\n-----")
    clear_output(wait = True) # to replace output with new one (instead of printing many outputs)

Now processing 1852th airport.
-----


In [10]:
len(airport_distance)

1853

In [11]:
# Add airport distance in metadata
metadata["airportDistance_km"] = airport_distance

# Preview the dataframe
metadata.head()

,index,name,categories,primaryCategories,address,city,province,latitude,longitude,websites,airportDistance_km
0,0,Rancho Valencia Resort Spa,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,5921 Valencia Cir,Rancho Santa Fe,CA,32.990959,-117.186136,http://www.ranchovalencia.com,14.308848
1,3,Aloft Arundel Mills,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,7520 Teague Rd,Hanover,MD,39.155929,-76.716341,http://www.starwoodhotels.com/alofthotels/prop...,4.668332
2,9,Hampton Inn Suites PortlandVancouver,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,315 SE Olympia Dr,Vancouver,WA,45.619212,-122.525196,http://hamptoninn3.hilton.com/en/hotels/washin...,6.591900
3,15,Hotel Phillips,"Hotels,Caterers,Hotels and motels,Hotel,Restau...",Accommodation & Food Services,106 W 12th St,Kansas City,MO,39.100119,-94.584701,http://curiocollection3.hilton.com/en/hotels/m...,2.670645
4,20,The Inn at Solvang,"Bed Breakfasts,Bed Breakfast,Hotels Motels,Hotel",Accommodation & Food Services,10611 Standing Stone Rd,Huntingdon,PA,40.527478,-77.969763,http://www.solvang.com,3.781817
